In [14]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import regionmask
import pandas as pd
from geopy.geocoders import Nominatim
from matplotlib.colors import SymLogNorm
from matplotlib.pyplot import cm

import xesmf as xe
#from pykrige.ok import OrdinaryKriging
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import dask
import utils

import geopandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#import data from:
### Gallagher, Kevin P. (2021), “China’s Global Energy Finance,” Global Development Policy Center, Boston University.
### Gallagher, Kevin P., Li, Zhongshu, Chen, Xu, Ma, Xinyue (2019), “China’s Global Power Database,” Global Development Policy Center, Boston University.

In [3]:
CGP_df = pd.read_excel('BU_data/CGP-Database_2020-1-2.xlsx')

In [4]:
CGP_df = CGP_df.loc[CGP_df['Technology'] == 'Coal']

In [5]:
#CGEF_df['Country'] = CGEF_df['Country'].str.lower()
CGP_df['Country'] = CGP_df['Country'].str.lower()

In [6]:
## https://pubs.acs.org/doi/abs/10.1021/es3003684 Table S1 
EF = np.exp(-3.64) #g/kg BC/coal

## https://www.nap.edu/read/9736/chapter/8 
HHF = 22.51 #GJ/t

#conversion factors
GJ_to_MwH = .28

Mw_to_MwH = 24 #daily

ton_to_kg = 0.001 #metric tons

MW_gpDay = Mw_to_MwH/GJ_to_MwH/HHF*ton_to_kg*EF #g/day


In [7]:
MW_gpDay*365

0.03648704276502928

In [8]:
years = 50
coal_year_range = np.arange(0,41)

In [9]:
## Add time dimension
length_simulation = years*365

time_array = np.arange(0, length_simulation)

In [10]:
CGP_df = CGP_df.loc[CGP_df['Region'] == 'Southeast Asia']

In [11]:
CGP_df.loc[:,'BC (g/day)'] = CGP_df['Capacity (MW)']*MW_gpDay

In [12]:
CGP_df.loc[:,'BC (g/day)'] 

4      0.069975
37     0.123956
39     0.062478
47     0.019993
91     0.009996
103    0.042585
119    0.065977
169    0.124456
170    0.119957
171    0.124456
214    0.119957
215    0.059979
216    0.059979
235    0.029989
236    0.098965
241    0.039986
247    0.150447
272    0.009996
323    0.059979
342    0.059979
346    0.124156
371    0.021992
406    0.062978
409    0.065977
416    0.043984
418    0.009996
423    0.094466
427    0.209925
439    0.040486
444    0.029989
452    0.063178
486    0.026990
487    0.013495
488    0.029989
507    0.029989
508    0.069975
515    0.012995
526    0.022392
527    0.016194
529    0.119957
532    0.059979
571    0.123956
572    0.124356
575    0.119957
Name: BC (g/day), dtype: float64

In [11]:
CGP_df.columns = CGP_df.columns.str.replace(' ', '_')

In [12]:
CGP_df.loc[CGP_df.Year_of_Commission == 'Pending', 'Year_of_Commission'] = 9999

In [13]:
CGP_df['CO2_weighted_capacity_1000tonsperMW'] = CGP_df['Estimated_Annual_CO2_Emission_from_Power_Generation_(1000_ton)']/CGP_df['Capacity_(MW)']

In [14]:
CGP_op = CGP_df.loc[CGP_df['Project_Status'] == 'In Operation']

In [44]:
print('Emis data prepped and loaded')

Emis data prepped and loaded


# Create Scenario

In [16]:
#######in progress retiring by year after co2#########
def early_retirement_by_CO2_year(year_early, df, CO2_val, time_array, shutdown_years):
    ''' df must have a variable 'Year_of_Commission' describing when the plant was comissioned, and 'BC_(g/day)' for BC emissions in g/day'''
    min_comission_yr = df['Year_of_Commission'].min()
    shutdown_days = shutdown_years*365
    E = np.zeros(len(time_array))
        #print(year_comis)
    #print(CGP_op.loc[CGP_op.Year_of_Commission == year_comis]['BC_(g/day)'].sum())
    test_array = np.where(time_array <= year_early*365, True, False)
    #plt.plot(test_array)
    E += test_array* df.loc[df.CO2_weighted_capacity_1000tonsperMW >= CO2_val]['BC_(g/day)'].sum()
        #fig, ax = plt.subplots()
        #plt.plot(E[year])
    for year_comis in np.arange(min_comission_yr, df['Year_of_Commission'].max()):
        #print(year_comis)
        #print(CGP_op.loc[CGP_op.Year_of_Commission == year_comis]['BC_(g/day)'].sum())
        test_array = np.where((time_array <= (year_comis-min_comission_yr)*365 + shutdown_days), True, False)
        #fig, ax = plt.subplots()
        #plt.plot(test_array* df.loc[(df.CO2_weighted_capacity_1000tonsperMW < CO2_val) & (df.Year_of_Commission == year_comis)]['BC_(g/day)'].sum())
        E += test_array* df.loc[(df.CO2_weighted_capacity_1000tonsperMW < CO2_val) & (df.Year_of_Commission == year_comis)]['BC_(g/day)'].sum()
        #E[year] += (time_array>=0) * df.loc[df.CO2_weighted_capacity_1000tonsperMW < CO2_val]['BC_(g/day)'].sum()
        #plt.plot(E)

    
    return(E)



In [18]:
percent = np.arange(1,101)

In [19]:
E_CO2_all_opts = {}
for year in coal_year_range:
    E_CO2_all_opts[year] = {}
    for r in percent:
        E_CO2_all_opts[year][r] = early_retirement_by_CO2_year(year, CGP_op, np.percentile(CGP_op['CO2_weighted_capacity_1000tonsperMW'],r), time_array, 40)
print('Emis profiiles created')

# Convolve with G

In [20]:
ds_area = xr.open_dataset('/net/fs11/d0/emfreese/GCrundirs/IRF_runs/RRTMG_pulse/SEA/Jan/mod_output/GEOSChem.SpeciesConc.20160101_0000z.nc4', engine = 'netcdf4')

In [21]:
#import the green's function and set our time step
G = xr.open_dataarray('Outputs/G_SEA_Jan_BC_total.nc4')
dt = 1 #day

In [22]:
G_lev0 = G.isel(lev = 0).sel().compute()
print('G prepped')

In [23]:
C_out = {}
C_sum = {}
for yr in coal_year_range:
    C_out[yr] = {}
    C_sum[yr] = {}
    for pc in percent:    
        C_out[yr][pc] = utils.convolve_single_lev(G_lev0 , E_CO2_all_opts[yr][pc])
        C_sum[yr][pc] = C_out[yr][pc].sum(dim = 's')

CPU times: user 2.39 s, sys: 278 ms, total: 2.67 s
Wall time: 2.67 s


In [24]:
country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110
countries = ['India','China','Indonesia','Malaysia','Vietnam','Australia', 'Cambodia','Myanmar', 'Laos','Philippines','Nepal','Bangladesh','Thailand','Bhutan']
print('countries uploaded')

In [40]:
loc_df = {}
for c in countries:
    loc_df[c] = pd.DataFrame(columns = percent, index = coal_year_range)
    mask = country_mask.mask(C_sum[1][10], lon_name = 'lon', lat_name = 'lat')
    contiguous_mask = ~np.isnan(mask)& (mask == country_mask.map_keys(c))
    for yr in coal_year_range:
        for pc in percent:
            print(yr,pc)
            loc_df[c][pc].loc[yr] = C_sum[yr][pc].where(contiguous_mask).weighted(ds_area['area']).mean().values
            print(c + 'done')
            loc_df[c].to_csv(f'Outputs/{c}_co2_year_shutdown.csv')
            print(c + 'saved')